<a href="https://colab.research.google.com/github/dongjaekim-hail/open-source-AI/blob/main/kaggle_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
import os
os.environ["KAGGLE_USERNAME"]="yourusername"
os.environ["KAGGLE_KEY"]="yourapikey"

In [3]:
!kaggle competitions list

ref                                                                                deadline             category                reward  teamCount  userHasEntered  
---------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2      2025-04-01 23:59:00  Featured         2,117,152 Usd        281           False  
https://www.kaggle.com/competitions/arc-prize-2024                                 2024-11-10 23:59:00  Featured         1,100,000 Usd       1320           False  
https://www.kaggle.com/competitions/gemma-language-tuning                          2025-01-15 00:59:00  Analytics          150,000 Usd          0           False  
https://www.kaggle.com/competitions/jane-street-real-time-market-data-forecasting  2025-01-13 23:59:00  Featured           120,000 Usd        845           False  
https://www.kagg

In [4]:
!kaggle competitions download -c osai-project

 98% 75.0M/76.7M [00:04<00:00, 28.7MB/s]
100% 76.7M/76.7M [00:04<00:00, 16.3MB/s]


In [5]:
!unzip osai-project.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/39280.png         
  inflating: train/39281.png         
  inflating: train/39285.png         
  inflating: train/39287.png         
  inflating: train/39401.png         
  inflating: train/39416.png         
  inflating: train/39420.png         
  inflating: train/39421.png         
  inflating: train/39427.png         
  inflating: train/39452.png         
  inflating: train/39465.png         
  inflating: train/39470.png         
  inflating: train/39472.png         
  inflating: train/39475.png         
  inflating: train/39482.png         
  inflating: train/39504.png         
  inflating: train/39524.png         
  inflating: train/39526.png         
  inflating: train/39540.png         
  inflating: train/39541.png         
  inflating: train/39542.png         
  inflating: train/39547.png         
  inflating: train/39560.png         
  inflating: train/39562.png         
  inflating: train/39576.png         
  inflating: t

In [6]:
!ls test

01285.png  13572.png  26107.png  37916.png  50921.png  63017.png  75962.png  89473.png
01397.png  13574.png  26437.png  37941.png  50981.png  63075.png  75981.png  89476.png
01425.png  13587.png  26457.png  37942.png  51024.png  63082.png  76012.png  89624.png
01562.png  13672.png  26481.png  37950.png  51346.png  63142.png  76120.png  89647.png
01567.png  13702.png  26590.png  37962.png  51396.png  63152.png  76490.png  89657.png
01594.png  13706.png  26705.png  38056.png  51467.png  63172.png  76492.png  89673.png
01674.png  13724.png  26731.png  38061.png  51670.png  63195.png  76540.png  89701.png
01723.png  13782.png  26754.png  38097.png  51672.png  63270.png  76803.png  89765.png
01735.png  13825.png  26785.png  38120.png  51806.png  63287.png  76830.png  90132.png
01762.png  13947.png  26813.png  38125.png  51920.png  63425.png  76853.png  90163.png
01834.png  13972.png  26837.png  38150.png  51926.png  63542.png  76942.png  90173.png
01874.png  14056.png  26915.png  38261.png 

In [7]:
import pandas as pd # csv 등 테이블 형태의 데이터를 처리하기 좋은 library

train = pd.read_csv('./train/text_label.csv')
test  = pd.read_csv('./test/text_label.csv')
val   = pd.read_csv('./val/text_label.csv')
sol   = pd.read_csv('./solution_sample.csv')

In [8]:
# 예시로 어떻게 들어가있는지 확인
train[:5]

,id,img,label,text
0,42953,train/42953.png,0,its their character not their color that matters
1,13894,train/13894.png,0,putting bows on your pet
2,82403,train/82403.png,0,"everybody loves chocolate chip cookies, even h..."
3,16952,train/16952.png,0,go sports! do the thing! win the points!
4,76932,train/76932.png,0,fine you're right. now can we fucking drop it?


In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision.transforms import ToTensor, ToPILImage
from tqdm import tqdm, trange
import numpy as np
to_tensor = ToTensor()
to_pil    = ToPILImage()


# 데이터셋을 불러오는 구조 만들기
class KaggleDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): Contains 'img', 'text', and 'label' columns.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.dataframe = dataframe
        self.transform = transform if transform else ToTensor()

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['img']
        text = self.dataframe.iloc[idx]['text']
        label = self.dataframe.iloc[idx]['label']

        # Load image and apply transform
        image = Image.open(img_path).convert("RGB")  # Ensure it's 3 channels
        image = self.transform(image)

        return image, text, label


In [14]:
# 데이터 불러오기
train_dataset = KaggleDataset(dataframe=train)  # 이미지 텍스트 레이블 칼럼 불러오는 데이터세트로 구성 데이터로더로 잘 받을 수 있는 형태
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)  # 데이터로더로 만들어서 필요할떄 하나씩 불러오도록

test_dataset = KaggleDataset(dataframe=test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

val_dataset = KaggleDataset(dataframe=val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


max_epoch = 1 # 50

for epoch in trange(max_epoch):
    count = 0
    progress_bar = tqdm(train_loader)
    for batch in progress_bar:
        images, texts, y_true = batch
        # 여기에 neural network 로 label 예측하는 부분이 필요하겠죠?
        y_pred = torch.randint(0, 2, (len(y_true),)) # 여기서는 예시니까 그냥 랜덤하게 하는걸로...

        # 정확도 계산
        accuracy = (y_pred == y_true).float().mean()
        count += 1

        progress_bar.set_description(f"TRAIN {count} - Accuracy: {accuracy:.4f}")

    # evaluation data
    count = 0
    progress_bar = tqdm(val_loader)
    for batch in progress_bar:
        images, texts, y_true = batch

        y_pred = torch.randint(0, 2, (len(y_true),)) # 여기서는 예시니까 그냥 랜덤하게

        accuracy = (y_pred == y_true).float().mean()
        count += 1

        progress_bar.set_description(f"EVAL {count} - Accuracy: {accuracy:.4f}")

TRAIN 211 - Accuracy: 0.4583: 100%|██████████| 211/211 [00:06<00:00, 31.30it/s]

100%|██████████| 1/1 [00:07<00:00,  7.70s/it]


In [15]:
resulting_label = []
progress_bar = tqdm(test_loader)
for batch in progress_bar:
    images, texts, labels = batch

    y_pred = torch.randint(0, 2, (len(texts),)) # 여기서는 예시니까 그냥 랜덤하게
    resulting_label.extend(y_pred.tolist())


100%|██████████| 50/50 [00:01<00:00, 41.18it/s]


In [16]:
sol

,id,label
0,57382,NaN
1,96137,NaN
2,43157,NaN
3,73851,NaN
4,10542,NaN
...,...,...
1588,6947,NaN
1589,87064,NaN
1590,24850,NaN
1591,39074,NaN


In [17]:
sol['label'] = resulting_label

In [18]:
sol

,id,label
0,57382,0
1,96137,1
2,43157,1
3,73851,1
4,10542,1
...,...,...
1588,6947,1
1589,87064,1
1590,24850,1
1591,39074,1


In [19]:
sol.to_csv('submission_tutorial.csv', index=False)

In [20]:
!kaggle competitions submit -c osai-project  -f ./submission_tutorial.csv -m "Submission via API"

100% 12.3k/12.3k [00:01<00:00, 8.81kB/s]
Successfully submitted to [오픈소스AI] 컴퓨터비전+텍스트분석 캐글 프로젝트

In [21]:
!kaggle competitions submissions -c osai-project

fileName                 date                 description         status    publicScore  privateScore  
-----------------------  -------------------  ------------------  --------  -----------  ------------  
submission_tutorial.csv  2024-10-31 08:16:43  Submission via API  complete  0.504                      


In [22]:
!kaggle competitions  leaderboard osai-project --show

  teamId  teamName     submissionDate       score  
--------  -----------  -------------------  -----  
12859134  Dongjae Kim  2024-10-31 08:16:43  0.504  
